In [ ]:
# Практична робота 3 Винник Михайло ФБ-52мп

## Завдання 1: Повнозв'язані нейронні мережі (FNN)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Встановлюю фіксований seed для відтворюваності результатів
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
try:
    df = pd.read_csv('earthquake_data_tsunami.csv')
    print("Дані успішно завантажено!")
except FileNotFoundError:
    print("Помилка: Файл 'earthquake_data_tsunami.csv' не знайдено.")

# Підготовка даних
df = df.fillna(0) # Заповнення пропусків
X = df.drop(columns=['tsunami'])
y = df['tsunami']

# Розбиття на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабування
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Дані успішно завантажено!


In [ ]:
# Архітектура моделі
model_fnn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),  # Запобігає перенавчанню
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid') # sigmoid для бінарної класифікації (0 або 1)
])

model_fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання
print("Початок навчання...")
history_fnn = model_fnn.fit(
    X_train_scaled, y_train, 
    epochs=30, 
    batch_size=16, 
    validation_split=0.2, 
    verbose=1
)

Початок навчання...
Epoch 1/30


c:\Users\misha\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6320 - loss: 0.6530 - val_accuracy: 0.7840 - val_loss: 0.5590
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7880 - loss: 0.4904 - val_accuracy: 0.8160 - val_loss: 0.4480
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8160 - loss: 0.4144 - val_accuracy: 0.8320 - val_loss: 0.4095
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8360 - loss: 0.3795 - val_accuracy: 0.8320 - val_loss: 0.3934
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8320 - loss: 0.3685 - val_accuracy: 0.8240 - val_loss: 0.3864
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8440 - loss: 0.3538 - val_accuracy: 0.8320 - val_loss: 0.3774
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8360 - loss: 0.3321 - val_accuracy: 0.8400 - val_loss: 0.3697
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8500 - loss: 0.3233 - val_accuracy: 0.8320 - val_loss: 0.3648
Ep

In [ ]:
# Оцінка на тестових даних
loss, acc = model_fnn.evaluate(X_test_scaled, y_test, verbose=0)
y_pred = (model_fnn.predict(X_test_scaled) > 0.5).astype(int)

print(f"\nТочність нейромережі: {acc:.4f}")
print("\nЗвіт класифікації:")
print(classification_report(y_test, y_pred))

# Порівняння з Random Forest (як у лабі 1)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)
print(f"Точність Random Forest (для порівняння): {rf.score(X_test_scaled, y_test):.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Точність нейромережі: 0.8471

Звіт класифікації:
              precision    recall  f1-score   support

           0       0.89      0.85      0.87        91
           1       0.80      0.85      0.82        66

    accuracy                           0.85       157
   macro avg       0.84      0.85      0.84       157
weighted avg       0.85      0.85      0.85       157

Точність Random Forest (для порівняння): 0.9299


## Завдання 2: Згорткові нейронні мережі (CNN)

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'intel_images/seg_train' 
test_dir = 'intel_images/seg_test'

img_size = (150, 150)
batch_size = 32

# Генератори з аугментацією (для кращого навчання)
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

try:
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
except:
    print("Не знайдено зображень. Перевірте шлях до папки!")

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [13]:
num_classes = train_generator.num_classes

model_cnn = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# навчання (зменшив кількість епох для швидкості, можна поставити більше)
history_cnn = model_cnn.fit(train_generator, epochs=5, validation_data=test_generator)

c:\Users\misha\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 116s 259ms/step - accuracy: 0.5980 - loss: 1.0536 - val_accuracy: 0.7293 - val_loss: 0.7323
Epoch 2/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 111s 253ms/step - accuracy: 0.7577 - loss: 0.6570 - val_accuracy: 0.8007 - val_loss: 0.5667
Epoch 3/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 108s 245ms/step - accuracy: 0.8047 - loss: 0.5350 - val_accuracy: 0.7967 - val_loss: 0.5717
Epoch 4/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 108s 246ms/step - accuracy: 0.8347 - loss: 0.4521 - val_accuracy: 0.8403 - val_loss: 0.4701
Epoch 5/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 111s 254ms/step - accuracy: 0.8658 - loss: 0.3742 - val_accuracy: 0.8437 - val_loss: 0.4526


In [ ]:
from tensorflow.keras.applications import VGG16

# Завантажуємо VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False # заморожуємо ваги

# додаю свої шари
model_tl = keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model_tl.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Навчання Transfer Learning моделі...")
history_tl = model_tl.fit(train_generator, epochs=5, validation_data=test_generator)

Навчання Transfer Learning моделі...
Epoch 1/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 439s 994ms/step - accuracy: 0.7991 - loss: 0.5486 - val_accuracy: 0.8510 - val_loss: 0.3748
Epoch 2/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 425s 967ms/step - accuracy: 0.8459 - loss: 0.4140 - val_accuracy: 0.8683 - val_loss: 0.3521
Epoch 3/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 419s 955ms/step - accuracy: 0.8648 - loss: 0.3675 - val_accuracy: 0.8763 - val_loss: 0.3282
Epoch 4/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 419s 954ms/step - accuracy: 0.8696 - loss: 0.3513 - val_accuracy: 0.8817 - val_loss: 0.3230
Epoch 5/5
439/439 ━━━━━━━━━━━━━━━━━━━━ 420s 956ms/step - accuracy: 0.8746 - loss: 0.3367 - val_accuracy: 0.8800 - val_loss: 0.3284


## Завдання 3: Класифікація тексту (NLP)

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 10000  # Кількість слів
maxlen = 200          # Довжина відгуку

print('Завантаження даних...')
(input_train, y_train_nlp), (input_test, y_test_nlp) = imdb.load_data(num_words=max_features) # датасет IMDB Movie Reviews

# вирівнювання довжини
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)

Завантаження даних...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
model_lstm = keras.Sequential([
    layers.Embedding(max_features, 32),
    layers.LSTM(32),
    layers.Dense(1, activation='sigmoid')
])

model_lstm.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

print("Навчання LSTM...")
history_lstm = model_lstm.fit(input_train, y_train_nlp, epochs=5, batch_size=128, validation_split=0.2)

Навчання LSTM...
Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 15s 83ms/step - accuracy: 0.6406 - loss: 0.6186 - val_accuracy: 0.7680 - val_loss: 0.5024
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.8231 - loss: 0.4049 - val_accuracy: 0.8592 - val_loss: 0.3407
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 16s 99ms/step - accuracy: 0.8665 - loss: 0.3260 - val_accuracy: 0.7834 - val_loss: 0.4693
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - accuracy: 0.8777 - loss: 0.3020 - val_accuracy: 0.8406 - val_loss: 0.3661
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.8949 - loss: 0.2670 - val_accuracy: 0.8624 - val_loss: 0.3400


In [ ]:
import os
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers, models
glove_dir = 'glove.6B.100d.txt'
embedding_dim = 100
max_features = 10000

# завантажую словник GloVe
embeddings_index = {}
if os.path.exists(glove_dir):
    print("Завантаження GloVe вектори...")
    with open(glove_dir, encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f"Знайдено {len(embeddings_index)} векторів слів.")

    # створюю матрицю з правильним зсувом індексів
    word_index = imdb.get_word_index()
    embedding_matrix = np.zeros((max_features, embedding_dim))
    
    hits = 0
    misses = 0
    
    for word, i in word_index.items():
        # Враховуємо зсув +3, який робить imdb.load_data()
        idx = i + 3 #Keras використовує індекси: 0 (padding), 1 (start), 2 (OOV). Тому реальні слова починаються з індексу 3.
        if idx < max_features:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[idx] = embedding_vector
                hits += 1
            else:
                misses += 1
    
    print(f"Конвертовано слів: {hits}, Не знайдено в GloVe: {misses}")

    # Модель
    model_glove = models.Sequential([
        layers.Embedding(max_features, 
                         embedding_dim, 
                         embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                         trainable=False), # False = не змінювати ваги GloVe
        layers.LSTM(32),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model_glove.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    print("Початок навчання моделі з GloVe...")
    history_glove = model_glove.fit(input_train, y_train_nlp, epochs=5, batch_size=128, validation_split=0.2)

else:
    print(f"УВАГА: Файл '{glove_dir}' не знайдено! Скачайте його з Kaggle.")

Завантаження GloVe вектори...
Знайдено 400000 векторів слів.
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Конвертовано слів: 9793, Не знайдено в GloVe: 203
Початок навчання моделі з GloVe...
Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 15s 78ms/step - accuracy: 0.6479 - loss: 0.6246 - val_accuracy: 0.7250 - val_loss: 0.5522
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.7344 - loss: 0.5389 - val_accuracy: 0.7502 - val_loss: 0.5152
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - accuracy: 0.7710 - loss: 0.4849 - val_accuracy: 0.7526 - val_loss: 0.5087
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.7944 - loss: 0.4417 - val_accuracy: 0.7958 - val_loss: 0.4465
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 70ms/step - accuracy: 0.8106 - loss: 0.4147 - val_accuracy: 0.8202 - val_loss: 0.4093
